In [20]:
# # install deps
# %pip install azure-cognitiveservices-speech
# %pip install openai
# # -----------------
# # %pip install azure-identity


In [21]:
# config
inc='inc1'

In [22]:
# read files form data_dir/input_files directory and create a list of filepaths
def create_file_hm(inc):
    import os
    input_dir=f'./data_dir/input_files/{inc}'
    print(f"Files in the directory: {input_dir}")
    files = os.listdir(input_dir)
    # Filtering only the files.
    # files = [f for f in files if os.path.isfile(input_dir+'/'+f)]
    files = [f for f in files]
    files=[f'{input_dir}/{f}' for f in files]
    print(*files, sep="\n")
    # fd={}
    # for f in files:
    #     fd[f]=open(f'{input_dir}/{f}')
    # print(fd)
    return files
files=create_file_hm(inc)

Files in the directory: ./data_dir/input_files/inc1
./data_dir/input_files/inc1/jackhammer.wav
./data_dir/input_files/inc1/mail2.txt
./data_dir/input_files/inc1/harvard.wav
./data_dir/input_files/inc1/mail1.txt


In [23]:
# processing text files
def proc_txt_data(files):
    txt_files=[f for f in files if '.txt' in f]
    txt_data=""
    for tf in txt_files:
        print(f"reading {tf} file----------------")
        with open(tf) as f:
            txt_data+='\n'.join(f.readlines())
    print(txt_data)
    return txt_data
txt_data=proc_txt_data(files)

reading ./data_dir/input_files/inc1/mail2.txt file----------------
reading ./data_dir/input_files/inc1/mail1.txt file----------------
this mail contains summary about the incident 34567


In [24]:
# load azure keys
import json
with open('../keys/azure.json') as json_file:
    azure_keys = json.load(json_file)
print(azure_keys.keys())


dict_keys(['stt_key1', 'stt_key2', 'openai_key1', 'openai_key2'])


In [25]:
# extract data from azure speech to text api : convert speech to text
import sys
# %pip install moviepy
# from moviepy.editor import *

# %pip install azure==4.0.0
import azure.cognitiveservices.speech as speechsdk


def proc_video_data(files):
    aud_files=[f for f in files if '.mp4' in f or '.wav' in f]
    
    aud_data=""
    for af in aud_files:
        speech_key, service_region = azure_keys['stt_key1'], "eastus"
        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
        audio_config = speechsdk.audio.AudioConfig(filename=af)
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
        result = speech_recognizer.recognize_once()
        print(result.text)
        aud_data+="\n"+result.text
    return aud_data

aud_data=proc_video_data(files)

The stale smell of old beer lingers.
The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham tacos. Al pastora are my favorite. A zestful food is the hot cross bun.


In [34]:
# create summary from azure open ai api
def generate_summary(content):
    print("content to summarize----------------\n",content)
    prompt=f""" 
    Generate a summary of our team meeting, highlighting key action items, decisions made, and deadlines set in key points. 
    #meeting transcript start:
    {content}
    #meeting transcript end
    """
    from openai import AzureOpenAI
    # from azure.identity import DefaultAzureCredential, get_bearer_token_provider
    deployment = "red-gpt"
    endpoint="https://genai-warriors-openai-test.openai.azure.com/"
    # token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        # azure_ad_token_provider=token_provider,
        api_version="2024-02-01",
        api_key=azure_keys['openai_key1']
    )
    completion = client.completions.create(
        model=deployment,
        prompt=prompt,


        temperature=0.3,
        max_tokens=350,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    response=completion
    res=response.choices[0].text


    print("\n\nsummary--------------------------\n",res)
    return res
   

content=txt_data+aud_data
summary=generate_summary(content)

content to summarize----------------
 this mail contains summary about the incident 34567
The stale smell of old beer lingers.
The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham tacos. Al pastora are my favorite. A zestful food is the hot cross bun.


summary--------------------------
 
Key Action Items:
- Create a detailed report on incident 34567
- Address the stale smell of old beer in the office
- Schedule a team outing for a cold dip to restore health and zest
- Plan a team lunch with salt pickles and ham tacos
- Assign someone to research and make hot cross buns for the team

Decisions Made:
- The incident 34567 will be addressed in a report
- The team will address the stale smell of old beer in the office
- A team outing for a cold dip will be scheduled
- A team lunch with salt pickles and ham tacos will be planned
- Someone will research and make hot cross buns for the team

Deadlines